In [1]:
import pandas as pd

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d tongpython/cat-and-dog

Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
 96% 209M/218M [00:04<00:00, 40.4MB/s]
100% 218M/218M [00:04<00:00, 54.6MB/s]


In [6]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cat-and-dog.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [29]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [16]:
# Generator
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/train/training_set',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)

test_ds=keras.utils.image_dataset_from_directory(
    directory='/content/test/test_set',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)

Found 8005 files belonging to 2 classes.
Found 2023 files belonging to 2 classes.


In [18]:
# Normalise
def process(image, label):
  image = tf.cast(image/256. ,tf.float32)
  return image,label

train_df = train_ds.map(process)
test_df = train_ds.map(process)

In [23]:
# crete CNN Model

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.fit(train_df, epochs=10, validation_data=test_df)

Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 49s 156ms/step - accuracy: 0.5654 - loss: 0.7523 - val_accuracy: 0.6070 - val_loss: 0.6432
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 70s 143ms/step - accuracy: 0.6548 - loss: 0.6273 - val_accuracy: 0.7142 - val_loss: 0.5510
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 32s 106ms/step - accuracy: 0.7295 - loss: 0.5380 - val_accuracy: 0.7448 - val_loss: 0.5184
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 51s 145ms/step - accuracy: 0.7754 - loss: 0.4674 - val_accuracy: 0.8576 - val_loss: 0.3264
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 27s 108ms/step - accuracy: 0.8644 - loss: 0.3029 - val_accuracy: 0.8846 - val_loss: 0.2873
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 27s 106ms/step - accuracy: 0.9247 - loss: 0.1855 - val_accuracy: 0.9313 - val_loss: 0.1881
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 51s 144ms/step - accuracy: 0.9681 - loss: 0.0904 - val_accuracy: 0.9192 - val_loss: 0.2528
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 36s 145ms/step - accuracy: 0.9774 - loss: 0

In [30]:
import cv2

In [36]:
test_img = cv2.imread('/content/cat.jpeg')

In [37]:
test_img.shape

(148, 188, 3)

In [38]:
test_img = cv2.resize(test_img, (256, 256))

In [39]:
test_input = test_img.reshape((1, 256, 256, 3))

In [40]:
model.predict(test_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


array([[0.]], dtype=float32)